In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import casadi as ca
import time

In [2]:
def robot_move(T_start_map, T_frame2frame):
    # Translate then rotate
    # Translate in current body frame, then rotate once you arrive to the new location.
    
    # Result is in map frame
    return T_start_map @ T_frame2frame

In [3]:
def meas_points(T_map2body_map, points_map):
    # Parse T_map2body_map
    t_map = T_map2body_map[0:3, 3]
    R_map2body = T_map2body_map[0:3, 0:3]
    
    # points_map = np.array([
    #             [1, 2, 3],
    #             [-1, 2, 3],
    #             [-1, -2, 3],
    #             [1,-2, 3],
    #             [1, 2, -3],
    #             [-1, 2, -3],
    #             [-1, -2, -3],
    #             [1, -2, -3],
    #             ])
    # points_map = np.tile(points_map,(20,1))
    # points_map = np.array([
    #     [1,2,3],
    #     [0,0,0],
    # ])
    
    y_map = np.zeros([len(points_map), 3])
    y_body = np.zeros([len(points_map), 3])
    for lcv, point in enumerate(points_map):
        # Point - Body = vector from body to point in map frame
        y_map[lcv, :] = points_map[lcv, :] - t_map
        
        # Convert map frame measurements to body frame via R_map2body
        y_body[lcv, :] = np.linalg.inv(R_map2body) @ y_map[lcv, :]
    
    return y_body
    

In [4]:
def calc_T_frame2frame(t_change_body, euler_change_body):
    # Calculates an SE(3) element
    # t_change_body - (3,1), (1,3), or (3,) vector with xyz translation
    # euler_change_body - (3,1), (1,3), or (3,) vector with roll, pitch, yaw Euler angles for XYZ rotation sequence.
    
    R = euler2rot(euler_change_body[0], euler_change_body[1], euler_change_body[2])
    
    T = np.eye(4)
    
    T[0:3, 0:3] = R
    T[0, 3] = t_change_body[0]
    T[1, 3] = t_change_body[1]
    T[2, 3] = t_change_body[2]
    
    return T


In [5]:
def euler2rot(phi, theta, psi):
    # Assumes roll, pitch, yaw sequence (1-2-3)
    # phi - roll angle in radians
    # theta - pitch angle in radians
    # psi - yaw angle in radians
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(phi), -np.sin(phi)],
                   [0, np.sin(phi), np.cos(phi)],
                  ])
    Ry = np.array([[np.cos(theta), 0, np.sin(theta)],
                   [0, 1, 0],
                   [-np.sin(theta), 0, np.cos(theta)],
                  ])
    Rz = np.array([[np.cos(psi), -np.sin(psi), 0],
                   [np.sin(psi), np.cos(psi), 0],
                   [0, 0, 1],
                  ])
    
    R = Rx @ Ry @ Rz
    
    return R

def euler2rot_website(phi, theta, psi):
    # DO NOT USE!!!
    # Assumes roll, pitch, yaw sequence (1-2-3)
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(phi), np.sin(phi)],
                   [0, -np.sin(phi), np.cos(phi)],
                  ])
    Ry = np.array([[np.cos(theta), 0, -np.sin(theta)],
                   [0, 1, 0],
                   [np.sin(theta), 0, np.cos(theta)],
                  ])
    Rz = np.array([[np.cos(psi), np.sin(psi), 0],
                   [-np.sin(psi), np.cos(psi), 0],
                   [0, 0, 1],
                  ])
    
    R = Rz @ Ry @ Rx
    
    return R

In [8]:
T0 = calc_T_frame2frame([0,0,0], [0, 0, 0])
T1 = calc_T_frame2frame([1,1,0], [0, 0, np.pi/2])
T2 = calc_T_frame2frame([0,0,0], [0, 0, np.pi/4])
T3 = calc_T_frame2frame([0,0,0], [np.pi/4, 0, 0])
T4 = calc_T_frame2frame([0.5,0,0], [0, 0, np.pi/4])

# Propogate movement
curr_pose = T0
curr_pose = robot_move(curr_pose, T4)
curr_pose = robot_move(curr_pose, T4)
curr_pose = robot_move(curr_pose, T4)

# points_map = np.array([
#     [1,2,3],
#     [0,0,0],
#     [2,1,1],
# ])
points_map = np.array([
                [1, 2, 3],
                [-1, 2, 3],
                [-1, -2, 3],
                [1,-2, 3],
                [1, 2, -3],
                [-1, 2, -3],
                [-1, -2, -3],
                [1, -2, -3],
                ])
points_map = np.tile(points_map,(20,1))

plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(points_map[:,0], points_map[:,1], points_map[:,2], color='g')
ax.set_xlabel('X Axis')
ax.set_ylabel('Y Axis')
ax.set_zlabel('Z Axis')
arrow_length = 0.5

# Plot origin or "map" tf frame
ax.quiver(0, 0, 0, 1, 0, 0, length=arrow_length, color='r')
ax.quiver(0, 0, 0, 0, 1, 0, length=arrow_length, color='g')
ax.quiver(0, 0, 0, 0, 0, 1, length=arrow_length, color='b')

# Plot body tf frame
t_map = curr_pose[0:3, 3]
R_map2body = curr_pose[0:3, 0:3]
x_axis = R_map2body @ np.array([[1,0,0]]).T
y_axis = R_map2body @ np.array([[0,1,0]]).T
z_axis = R_map2body @ np.array([[0,0,1]]).T
ax.quiver(t_map[0], t_map[1], t_map[2], x_axis[0], x_axis[1], x_axis[2], length=arrow_length, color='r')
ax.quiver(t_map[0], t_map[1], t_map[2], y_axis[0], y_axis[1], y_axis[2], length=arrow_length, color='g')
ax.quiver(t_map[0], t_map[1], t_map[2], z_axis[0], z_axis[1], z_axis[2], length=arrow_length, color='b')

plt.autoscale(False)
ax.set_aspect('equal','box')
plt.show()

print(np.round(curr_pose,4))
# print(np.round(meas_points(curr_pose, points_map),4))

[[-0.7071 -0.7071  0.      0.8536]
 [ 0.7071 -0.7071  0.      0.8536]
 [ 0.      0.      1.      0.    ]
 [ 0.      0.      0.      1.    ]]


In [62]:
R1 = euler2rot(0, 0, np.pi/2)
R2 = euler2rot(0, 0, -np.pi/4)
print(np.round(R1,4))
print(np.round(R2,4))

# test_p = np.array([1, 2, 3])

# print(np.round(R1@R2@test_p, 4))   # Intrinsic (uses newly tranformed axes)
# print(np.round(R2@R1@test_p, 4))   # Extrinsic (all axes are global)

# # We like intrinsic!!!

[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]
[[ 0.7071  0.7071  0.    ]
 [-0.7071  0.7071  0.    ]
 [ 0.      0.      1.    ]]
